# 03 · Ranker Lab — фичи, ранкер, OOF и тестовый скор 🔻

Цель: из пула кандидатов собрать фичи **time-safe**, обучить 1–2 ранкера (LightGBM LambdaRank / CatBoost Ranking), получить **OOF** и **test** предсказания, проверить срезы и сохранить артефакты.

Выходы:
`artifacts/recsys/ranker/<id>/<profile>/<run_id>/{X_rank_val.parquet, X_rank_test.parquet, oof.parquet, ranked.parquet, model.*, fi.csv, ranker_report.json}`

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, time, math, warnings, shlex, subprocess
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

from IPython.display import display, clear_output, HTML
import ipywidgets as W

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 140)

# Легкий импорт библиотек ранкинга
try:
    import lightgbm as lgb
except Exception as e:
    lgb = None
try:
    from catboost import CatBoostRanker, Pool
except Exception:
    CatBoostRanker = None
    Pool = None

REPO = Path.cwd()

def mem_gb(obj=None):
    try:
        import psutil
        if obj is None: return psutil.Process().memory_info().rss/(1024**3)
    except Exception:
        pass
    try:
        if hasattr(obj, "memory_usage"):
            return float(obj.memory_usage(deep=True).sum())/(1024**3)
        return float(np.array(obj).nbytes)/(1024**3)
    except Exception:
        return float("nan")

def save_json(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2))

def now_tag(): return datetime.now().strftime("%m%d_%H%M")

def art_paths(dataset_id, profile_path):
    prof = Path(profile_path).stem
    root = Path("artifacts")
    return dict(
        dataio     = root/"recsys"/"dataio"/dataset_id,
        profile    = root/"recsys"/"dataio"/dataset_id/prof,
        candidates = root/"recsys"/"candidates"/dataset_id/prof,
        ranker     = root/"recsys"/"ranker"/dataset_id/prof,
    )

def exists(p): 
    try: return Path(p).exists()
    except: return False

def floor_day(ts): 
    s = pd.to_datetime(ts, utc=True)
    return s.dt.floor("D")

def merge_asof_by(df_left, df_right, on, by, direction="backward"):
    """Упрощённая обертка для merge_asof с группировкой по 'by'."""
    l = df_left.sort_values([by, on])
    r = df_right.sort_values([by, on])
    return pd.merge_asof(l, r, on=on, by=by, direction=direction)

In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
STATE = dict(
    DATASET_ID   = "s5e11",
    PROFILE_PATH = "configs/recsys/profiles/gate.yaml",
    USE_CAND_LOCAL_QUOTA = False,
    K_IN   = 300,   # сколько берём кандидатов на ранкинг на query
    K_OUT  = 100,   # сколько оставляем в финальном ranked.parquet
    RANDOM_SEED = 42,
    JOBS   = -1,
)

CV = dict(
    CV_KIND   = "kfold_queries",   # kfold_queries | group_user | time
    N_SPLITS  = 5,
    EMBARGO   = "0D"
)

MODEL = dict(
    LIB           = "lightgbm",    # lightgbm | catboost
    OBJECTIVE     = "lambdarank",  # для LGBM: lambdarank; для CatBoost: YetiRankPairwise/PairLogitPairwise
    N_EST         = 400,
    LR            = 0.05,
    MAX_DEPTH     = 6,
    NUM_LEAVES    = 63,
    L2            = 1.0,
    MIN_DATA_LEAF = 20,
    FEATURE_FRACT = 0.9,
    BAGGING_FRACT = 0.8,
    EARLY_ROUNDS  = 50
)

FEATS = dict(
    use_source_feats    = True,
    use_pop_recency     = True,
    use_user_hist       = True,
    use_user_item_match = True,
    use_item_meta       = True,
    use_context         = True,
    use_cold_flags      = True,

    # параметры фич
    windows_days = [7, 30],
    user_last_k  = 5,
)

# --- Виджеты
w_ds    = W.Text(STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="240px"))
w_prof  = W.Text(STATE["PROFILE_PATH"], description="profile:", layout=W.Layout(width="520px"))
w_k_in  = W.IntText(STATE["K_IN"], description="K_IN:", layout=W.Layout(width="150px"))
w_k_out = W.IntText(STATE["K_OUT"], description="K_OUT:", layout=W.Layout(width="150px"))
w_seed  = W.IntText(STATE["RANDOM_SEED"], description="seed:", layout=W.Layout(width="150px"))
w_jobs  = W.IntText(STATE["JOBS"], description="jobs:", layout=W.Layout(width="150px"))
w_cand_local = W.Checkbox(STATE["USE_CAND_LOCAL_QUOTA"], description="use candidates_local_quota")

w_cv_kind = W.Dropdown(options=["kfold_queries","group_user","time"], value=CV["CV_KIND"], description="CV_KIND:")
w_cv_ns   = W.IntText(CV["N_SPLITS"], description="N_SPLITS:")
w_cv_emb  = W.Text(CV["EMBARGO"], description="EMBARGO:")

w_lib   = W.Dropdown(options=["lightgbm","catboost"], value=MODEL["LIB"], description="LIB:")
w_obj   = W.Dropdown(options=["lambdarank","pairwise","yeti"], value=MODEL["OBJECTIVE"], description="objective:")
w_nest  = W.IntText(MODEL["N_EST"], description="n_estimators:")
w_lr    = W.FloatText(MODEL["LR"], description="learning_rate:")
w_maxd  = W.IntText(MODEL["MAX_DEPTH"], description="max_depth:")
w_leaves= W.IntText(MODEL["NUM_LEAVES"], description="num_leaves:")
w_l2    = W.FloatText(MODEL["L2"], description="l2:")
w_minlf = W.IntText(MODEL["MIN_DATA_LEAF"], description="min_data_leaf:")
w_ff    = W.FloatText(MODEL["FEATURE_FRACT"], description="feature_fraction:")
w_bf    = W.FloatText(MODEL["BAGGING_FRACT"], description="bagging_fraction:")
w_es    = W.IntText(MODEL["EARLY_ROUNDS"], description="early_rounds:")

tog = dict(
    src   = W.Checkbox(FEATS["use_source_feats"], description="source"),
    pop   = W.Checkbox(FEATS["use_pop_recency"], description="pop/recency"),
    uhist = W.Checkbox(FEATS["use_user_hist"], description="user_hist"),
    uim   = W.Checkbox(FEATS["use_user_item_match"], description="user_item_match"),
    imeta = W.Checkbox(FEATS["use_item_meta"], description="item_meta"),
    ctx   = W.Checkbox(FEATS["use_context"], description="context"),
    cs    = W.Checkbox(FEATS["use_cold_flags"], description="cold_flags"),
)

w_windows = W.Text(",".join(map(str, FEATS["windows_days"])), description="windows_days:")
w_lastk   = W.IntText(FEATS["user_last_k"], description="user_last_k:")

btn_apply = W.Button(description="Применить параметры", button_style="success", layout=BTN_LAYOUT)
out_apply = W.Output()

def on_apply(_):
    with out_apply:
        clear_output()
        STATE.update(
            DATASET_ID=w_ds.value.strip(),
            PROFILE_PATH=w_prof.value.strip(),
            K_IN=int(w_k_in.value),
            K_OUT=int(w_k_out.value),
            RANDOM_SEED=int(w_seed.value),
            JOBS=int(w_jobs.value),
            USE_CAND_LOCAL_QUOTA=bool(w_cand_local.value)
        )
        CV.update(
            CV_KIND=w_cv_kind.value,
            N_SPLITS=int(w_cv_ns.value),
            EMBARGO=w_cv_emb.value.strip()
        )
        MODEL.update(
            LIB=w_lib.value,
            OBJECTIVE=w_obj.value,
            N_EST=int(w_nest.value),
            LR=float(w_lr.value),
            MAX_DEPTH=int(w_maxd.value),
            NUM_LEAVES=int(w_leaves.value),
            L2=float(w_l2.value),
            MIN_DATA_LEAF=int(w_minlf.value),
            FEATURE_FRACT=float(w_ff.value),
            BAGGING_FRACT=float(w_bf.value),
            EARLY_ROUNDS=int(w_es.value)
        )
        FEATS.update(
            use_source_feats=bool(tog["src"].value),
            use_pop_recency=bool(tog["pop"].value),
            use_user_hist=bool(tog["uhist"].value),
            use_user_item_match=bool(tog["uim"].value),
            use_item_meta=bool(tog["imeta"].value),
            use_context=bool(tog["ctx"].value),
            use_cold_flags=bool(tog["cs"].value),
            windows_days=[int(x) for x in w_windows.value.split(",") if x.strip()],
            user_last_k=int(w_lastk.value),
        )
        print("STATE:", json.dumps(STATE, indent=2, ensure_ascii=False))
        print("CV:", json.dumps(CV, indent=2, ensure_ascii=False))
        print("MODEL:", json.dumps(MODEL, indent=2, ensure_ascii=False))
        print("FEATS:", json.dumps(FEATS, indent=2, ensure_ascii=False))

W.VBox([
    W.HTML("<h3>Параметры</h3>"),
    W.HBox([w_ds, w_seed, w_jobs, w_k_in, w_k_out, w_cand_local], layout=ROW_LAYOUT),
    w_prof,
    W.HTML("<b>CV</b>"),
    W.HBox([w_cv_kind, w_cv_ns, w_cv_emb], layout=ROW_LAYOUT),
    W.HTML("<b>Модель</b>"),
    W.HBox([w_lib, w_obj, w_nest, w_lr, w_maxd, w_leaves], layout=ROW_LAYOUT),
    W.HBox([w_l2, w_minlf, w_ff, w_bf, w_es], layout=ROW_LAYOUT),
    W.HTML("<b>Фичи</b>"),
    W.HBox([tog["src"], tog["pop"], tog["uhist"], tog["uim"], tog["imeta"], tog["ctx"], tog["cs"]], layout=ROW_LAYOUT),
    W.HBox([w_windows, w_lastk], layout=ROW_LAYOUT),
    btn_apply, out_apply
])

In [ ]:
btn_check = W.Button(description="Загрузить входы и проверить покрытие", button_style="primary", layout=BTN_LAYOUT)
out_check = W.Output()

CACHE = dict()  # inter/items/queries/pairs/cands

def on_check(_):
    with out_check:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        p_inter = A["dataio"]/ "interactions_norm.parquet"
        p_items = A["dataio"]/ "items_norm.parquet"
        p_splits= A["dataio"]/ "splits.json"
        p_qv    = A["profile"]/ "queries_val.parquet"
        p_pv    = A["profile"]/ "pairs_val.parquet"
        p_qt    = A["profile"]/ "queries_test.parquet"
        cdir    = A["candidates"]
        p_cand  = cdir/"candidates_local_quota.parquet" if STATE["USE_CAND_LOCAL_QUOTA"] and exists(cdir/"candidates_local_quota.parquet") else cdir/"candidates.parquet"
        need = [p_inter,p_items,p_splits,p_qv,p_pv,p_qt,p_cand]
        ok=True
        for p in need:
            tag = "OK " if exists(p) else "MISS"
            print(tag, p)
            ok &= exists(p)
        if not ok:
            print("\n⚠️ Нет нужных артефактов — вернись в 01/02.")
            return
        inter = pd.read_parquet(p_inter)
        items = pd.read_parquet(p_items)
        splits = json.loads(Path(p_splits).read_text())
        qv = pd.read_parquet(p_qv)
        pv = pd.read_parquet(p_pv)
        qt = pd.read_parquet(p_qt)
        cands = pd.read_parquet(p_cand)

        # ограничим кандидатов до K_IN
        if "score" in cands.columns:
            cands = cands.sort_values(["query_id","score"], ascending=[True,False])
        cands = cands.groupby("query_id").head(STATE["K_IN"]).reset_index(drop=True)

        print("\ninter:", inter.shape, "| items:", items.shape, "| cands:", cands.shape)
        print("queries_val:", qv.shape, "pairs_val:", pv.shape, "queries_test:", qt.shape)

        # покрытие
        cov = cands["query_id"].nunique()/ max(1, qv["query_id"].nunique())
        print(f"coverage (queries_val with >=1 cand): {cov:.2%}")
        if cov < 0.97:
            print("⚠️ низкое покрытие кандидатов — вернись в 02, включи fallbacks.")

        # Кэш
        CACHE.update(inter=inter, items=items, splits=splits, qv=qv, pv=pv, qt=qt, cands=cands)

btn_check.on_click(on_check)
W.VBox([btn_check, out_check])

In [ ]:
btn_preagg = W.Button(description="Построить тайм-safe агрегаты (item/user по дням)", button_style="primary", layout=BTN_LAYOUT)
out_preagg = W.Output()

AGG = dict(item_day=None, item_day_cum=None, user_day=None, user_day_cum=None, user_events=None, ui_timeline=None)

def build_daily_cum(df, key_col, ts_col="ts"):
    tmp = df[[key_col, ts_col]].copy()
    tmp["day"] = floor_day(tmp[ts_col])
    g = tmp.groupby([key_col, "day"]).size().rename("cnt").reset_index()
    g = g.sort_values([key_col, "day"])
    g["cum_cnt"] = g.groupby(key_col)["cnt"].cumsum()
    return g  # колонки: key_col, day, cnt, cum_cnt

def build_user_events(df):
    # для merge_asof по последнему событию пользователя
    ev = df[["user_id","ts"]].copy().sort_values(["user_id","ts"]).reset_index(drop=True)
    ev["u_cum_cnt"] = ev.groupby("user_id").cumcount()+1
    return ev

def build_user_item_timeline(df):
    # для seen_before и times_seen_before по конкретному (user,item)
    tmp = df[["user_id","item_id","ts"]].copy().sort_values(["user_id","item_id","ts"])
    tmp["uid_item"] = (tmp["user_id"].astype("int64")<<32) + tmp["item_id"].astype("int64")
    tmp["ui_cum_cnt"] = tmp.groupby(["user_id","item_id"]).cumcount()+1
    return tmp[["uid_item","ts","ui_cum_cnt"]]

def on_preagg(_):
    with out_preagg:
        clear_output()
        inter = CACHE.get("inter")
        if inter is None:
            print("Сначала загрузите входы.")
            return
        t0=time.time()
        AGG["item_day"]      = build_daily_cum(inter.rename(columns={"item_id":"KEY"}), "KEY").rename(columns={"KEY":"item_id"})
        AGG["item_day_cum"]  = AGG["item_day"][ ["item_id","day","cum_cnt"] ].copy()
        AGG["user_day"]      = build_daily_cum(inter.rename(columns={"user_id":"KEY"}), "KEY").rename(columns={"KEY":"user_id"})
        AGG["user_day_cum"]  = AGG["user_day"][ ["user_id","day","cum_cnt"] ].copy()
        AGG["user_events"]   = build_user_events(inter)
        AGG["ui_timeline"]   = build_user_item_timeline(inter)
        dt=time.time()-t0
        print("✓ готово за", f"{dt:.1f}s")
        for k,v in AGG.items():
            if isinstance(v, pd.DataFrame):
                print(k, v.shape, "| mem:", f"{mem_gb(v):.3f} GB")
btn_preagg.on_click(on_preagg)
W.VBox([btn_preagg, out_preagg])

In [ ]:
def windows_to_list(ws): 
    return [int(x) for x in ws] if isinstance(ws, (list,tuple)) else [int(y) for y in str(ws).split(",")]

def item_pop_at_queries(cum_df, df_q, windows=(7,30), item_col="item_id", q_ts_col="query_ts"):
    """Возвращает DataFrame с колонками pop@W для каждого окна W. time-safe: cum_cnt(item, day-1) - cum_cnt(item, day-W-1)."""
    windows = windows_to_list(windows)
    q = df_q[[item_col, q_ts_col]].copy()
    q["day"] = floor_day(q[q_ts_col])
    out = q[[item_col, "day"]].copy()
    base = cum_df.copy()
    base = base.sort_values([item_col, "day"])
    for W in windows:
        right = out.rename(columns={"day":"day_right"})
        right["day_right"] = right["day_right"] - pd.Timedelta(days=1)
        right = pd.merge_asof(right.sort_values("day_right"), base, left_on="day_right", right_on="day", by=item_col, direction="backward")
        right = right.rename(columns={"cum_cnt":f"cum_r_{W}"}).drop(columns=["day"])
        left = out.rename(columns={"day":"day_left"})
        left["day_left"] = left["day_left"] - pd.Timedelta(days=W+1)
        left = pd.merge_asof(left.sort_values("day_left"), base, left_on="day_left", right_on="day", by=item_col, direction="backward")
        left = left.rename(columns={"cum_cnt":f"cum_l_{W}"}).drop(columns=["day"])
        m = pd.concat([out, right[[f"cum_r_{W}"]], left[[f"cum_l_{W}"]]], axis=1)
        m[f"item_pop@{W}"] = (m[f"cum_r_{W}"].fillna(0) - m[f"cum_l_{W}"].fillna(0)).astype("float32")
        out = pd.concat([out, m[[f"item_pop@{W}"]]], axis=1)
    return out.drop(columns=["day"])

def user_activity_at_queries(cum_df, df_q, windows=(7,30), user_col="user_id", q_ts_col="query_ts"):
    """User events count за окна W до query_ts."""
    windows = windows_to_list(windows)
    q = df_q[[user_col, q_ts_col]].copy()
    q["day"] = floor_day(q[q_ts_col])
    out = q[[user_col, "day"]].copy()
    base = cum_df.copy().sort_values([user_col,"day"])
    for W in windows:
        right = out.rename(columns={"day":"day_right"})
        right["day_right"] = right["day_right"] - pd.Timedelta(days=1)
        right = pd.merge_asof(right.sort_values("day_right"), base, left_on="day_right", right_on="day", by=user_col, direction="backward")
        right = right.rename(columns={"cum_cnt":f"u_cum_r_{W}"}).drop(columns=["day"])
        left = out.rename(columns={"day":"day_left"})
        left["day_left"] = left["day_left"] - pd.Timedelta(days=W+1)
        left = pd.merge_asof(left.sort_values("day_left"), base, left_on="day_left", right_on="day", by=user_col, direction="backward")
        left = left.rename(columns={"cum_cnt":f"u_cum_l_{W}"}).drop(columns=["day"])
        m = pd.concat([out, right[[f"u_cum_r_{W}"]], left[[f"u_cum_l_{W}"]]], axis=1)
        out[f"user_events@{W}"] = (m[f"u_cum_r_{W}"].fillna(0) - m[f"u_cum_l_{W}"].fillna(0)).astype("float32")
    return out.drop(columns=["day"])

def user_last_event_gap(user_events, df_q, user_col="user_id", q_ts_col="query_ts"):
    """Время с последнего пользовательского события до запроса (дни)."""
    ue = user_events.copy()
    # merge_asof по user_id: последнее событие ≤ query_ts
    m = merge_asof_by(df_q[[user_col, q_ts_col]].rename(columns={q_ts_col:"ts"}), ue.rename(columns={"ts":"ue_ts"}), on="ts", by=user_col, direction="backward")
    gap_days = ((m["ts"] - m["ue_ts"]).dt.total_seconds()/86400.0).fillna(1e6).astype("float32")
    return pd.DataFrame({user_col: df_q[user_col].values, q_ts_col: df_q[q_ts_col].values, "u_gap_days": gap_days})

def user_item_seen_features(ui_timeline, df_q, user_col="user_id", item_col="item_id", q_ts_col="query_ts"):
    """Признаки: видел ли пользователь item до запроса, сколько раз, давность (дни)."""
    t = ui_timeline.copy()
    q = df_q[[user_col,item_col,q_ts_col]].copy()
    q["uid_item"] = (q[user_col].astype("int64")<<32)+q[item_col].astype("int64")
    # merge_asof по (uid_item, ts)
    left = q.rename(columns={q_ts_col:"ts"})
    m = merge_asof_by(left[["uid_item","ts"]], t.rename(columns={"ts":"ui_ts"}), on="ts", by="uid_item", direction="backward")
    seen_before = m["ui_cum_cnt"].fillna(0).astype("int32")
    # давность
    rec_days = ((left["ts"] - m["ui_ts"]).dt.total_seconds()/86400.0).fillna(1e6).astype("float32")
    return pd.DataFrame({
        user_col: q[user_col].values,
        item_col: q[item_col].values,
        q_ts_col: q[q_ts_col].values,
        "ui_seen_before": (seen_before>0).astype("int8"),
        "ui_seen_cnt": seen_before.astype("int16"),
        "ui_recency_days": rec_days
    })

In [ ]:
btn_build_feats = W.Button(description="Собрать фичи для VAL/TEST", button_style="success", layout=BTN_LAYOUT)
out_build_feats = W.Output()

DATA = dict(X_val=None, y_val=None, group_val=None, X_test=None, group_test=None, feat_cols=None, meta=None)

def build_features(split="val"):
    """Собираем фичи, используя CACHE и AGG."""
    assert split in ("val","test")
    inter, items, cands = CACHE["inter"], CACHE["items"], CACHE["cands"]
    if split=="val":
        q = CACHE["qv"][ ["query_id","user_id","query_ts"] ].copy()
        pairs = CACHE["pv"][ ["query_id","item_id","label"] ].copy()
    else:
        q = CACHE["qt"][ ["query_id","user_id","query_ts"] ].copy()
        pairs = None

    # ограничим кандидатов текущими query
    cand = cands.merge(q[["query_id","user_id","query_ts"]], on="query_id", how="inner")
    # сорируем и обрезаем до K_IN (на всякий)
    if "score" in cand.columns:
        cand = cand.sort_values(["query_id","score"], ascending=[True, False])
    cand = cand.groupby("query_id").head(STATE["K_IN"]).reset_index(drop=True)

    # --- SOURCE фичи
    feats = []
    base = cand[["query_id","user_id","item_id","query_ts"]].copy()
    if FEATS["use_source_feats"]:
        sf = []
        if "source" in cand.columns:
            # one-hot источников + счетчик источников (если дубликаты были)
            src_dummies = pd.get_dummies(cand["source"], prefix="src").astype("int8")
            tmp = pd.concat([cand[["query_id","item_id"]], src_dummies], axis=1)
            g = tmp.groupby(["query_id","item_id"]).sum().reset_index()
            sf.append(g)
        if "score" in cand.columns:
            sc = cand[["query_id","item_id","score"]].copy()
            sc["score"] = sc["score"].astype("float32")
            sc = sc.groupby(["query_id","item_id"])["score"].max().reset_index().rename(columns={"score":"cand_score_max"})
            sf.append(sc)
        if "source_rank" in cand.columns:
            rk = cand[["query_id","item_id","source_rank"]].copy()
            rk["source_rank"] = rk["source_rank"].astype("int32")
            rk = rk.groupby(["query_id","item_id"])["source_rank"].min().reset_index().rename(columns={"source_rank":"src_rank_min"})
            sf.append(rk)
        if sf:
            s = sf[0]
            for t in sf[1:]:
                s = s.merge(t, on=["query_id","item_id"], how="outer")
            feats.append(s)

    # --- POP/RECENCY (time-safe через кумы)
    if FEATS["use_pop_recency"]:
        item_pop = item_pop_at_queries(AGG["item_day_cum"], base.rename(columns={"query_ts":"_qt"}).rename(columns={"_qt":"query_ts"}), windows=FEATS["windows_days"])
        pr = pd.concat([base[["query_id","item_id"]], item_pop.drop(columns=["item_id","query_ts"])], axis=1)
        feats.append(pr)

    # --- USER HIST (счётчики за окна + gap до последней активности)
    if FEATS["use_user_hist"]:
        ucnt = user_activity_at_queries(AGG["user_day_cum"], base.rename(columns={"query_ts":"_qt"}).rename(columns={"_qt":"query_ts"}), windows=FEATS["windows_days"])
        ugap = user_last_event_gap(AGG["user_events"], base.rename(columns={"query_ts":"_qt"}).rename(columns={"_qt":"query_ts"}))
        uh = base[["query_id"]].join(ucnt.drop(columns=["user_id","query_ts"]))
        uh = pd.concat([uh, ugap.drop(columns=["user_id","query_ts"])], axis=1)
        feats.append(pd.concat([base[["query_id","item_id"]], uh.drop(columns=["query_id"])], axis=1))

    # --- USER–ITEM match (видел/сколько/давно)
    if FEATS["use_user_item_match"]:
        uim = user_item_seen_features(AGG["ui_timeline"], base)
        uim2 = pd.concat([base[["query_id"]], uim.drop(columns=["user_id","item_id","query_ts"])], axis=1)
        feats.append(pd.concat([base[["query_id","item_id"]], uim2.drop(columns=["query_id"])], axis=1))

    # --- ITEM META (например brand/category/price, совпадения с недавними брендами можно добавить при желании)
    if FEATS["use_item_meta"]:
        join_cols = ["item_id"]
        meta_cols = [c for c in ["brand","category","price","created_ts"] if c in items.columns]
        im = items[join_cols+meta_cols].copy()
        if "created_ts" in im.columns:
            im["item_age_days"] = ((base["query_ts"].iloc[0] if len(base) else pd.Timestamp.utcnow()) - pd.to_datetime(im["created_ts"], utc=True)).dt.days.astype("float32") if im["created_ts"].notna().any() else 0.0
            im = im.drop(columns=["created_ts"])
        # one-hot для крупного brand/category — для экономии пропустим; оставим price + age
        im["price"] = im.get("price", pd.Series(0.0, index=im.index)).astype("float32")
        feats.append(base[["query_id","item_id"]].merge(im, on="item_id", how="left"))

    # --- CONTEXT
    if FEATS["use_context"]:
        ctx = base[["query_id","item_id","query_ts"]].copy()
        ctx["hour"] = ctx["query_ts"].dt.hour.astype("int16")
        ctx["dow"]  = ctx["query_ts"].dt.dayofweek.astype("int16")
        ctx = ctx.drop(columns=["query_ts"])
        feats.append(ctx)

    # --- COLD-flags (по train-периоду из splits)
    if FEATS["use_cold_flags"]:
        # Определяем train_end из splits.json
        t_train_end = pd.to_datetime(CACHE["splits"]["bounds"]["train_end"])
        tr = inter[inter["ts"]<=t_train_end]
        cold_item = ~base["item_id"].isin(tr["item_id"].unique())
        cold_user = ~base["user_id"].isin(tr["user_id"].unique())
        cf = base[["query_id","item_id"]].copy()
        cf["is_cold_item"] = cold_item.astype("int8").values
        cf["is_cold_user"] = cold_user.astype("int8").values
        feats.append(cf)

    # --- merge всех фич
    X = base[["query_id","item_id"]].copy()
    for f in feats:
        X = X.merge(f, on=["query_id","item_id"], how="left")
    # заполнение
    for c in X.columns:
        if c in ("query_id","item_id"): continue
        if pd.api.types.is_integer_dtype(X[c]): X[c] = X[c].fillna(0)
        else: X[c] = X[c].fillna(0.0)

    # label и группы
    if split=="val":
        y = pairs.merge(X[["query_id","item_id"]], on=["query_id","item_id"], how="right").fillna({"label":0})["label"].astype("int8")
        group = X["query_id"].value_counts().sort_index()
        return X, y, group
    else:
        group = X["query_id"].value_counts().sort_index()
        return X, None, group

def on_build_feats(_):
    with out_build_feats:
        clear_output()
        if not CACHE or not AGG["item_day_cum"] is not None:
            print("Сначала загрузите входы и постройте агрегаты.")
            return
        t0=time.time()
        Xv, yv, gv = build_features("val")
        Xt, _,  gt = build_features("test")
        DATA.update(X_val=Xv, y_val=yv, group_val=gv, X_test=Xt, group_test=gt)
        feat_cols = [c for c in Xv.columns if c not in ("query_id","item_id")]
        DATA["feat_cols"]=feat_cols
        print("VAL:", Xv.shape, "| TEST:", Xt.shape, "| #feats:", len(feat_cols), "| mem(Xv):", f"{mem_gb(Xv):.3f} GB")
        print("Примеры фич:", feat_cols[:20])
        # сохраним промежуточные наборы
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_id = now_tag()
        run_dir = A["ranker"]/run_id
        run_dir.mkdir(parents=True, exist_ok=True)
        Xv.to_parquet(run_dir/"X_rank_val.parquet", index=False)
        Xt.to_parquet(run_dir/"X_rank_test.parquet", index=False)
        DATA["meta"] = dict(run_id=run_id, run_dir=str(run_dir))
        print("Saved to:", run_dir)

btn_build_feats.on_click(on_build_feats)
W.VBox([btn_build_feats, out_build_feats])

In [ ]:
from sklearn.model_selection import KFold, GroupKFold

FOLDS = dict(pairs=[])

def make_folds(kind, n_splits, Xv, cv_meta=None):
    # группы по query_id для ранкинга
    qids = Xv["query_id"].values
    uniq_q = np.unique(qids)
    if kind=="kfold_queries":
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=STATE["RANDOM_SEED"])
        return [(np.isin(qids, uniq_q[tr]), np.isin(qids, uniq_q[va])) for tr,va in kf.split(uniq_q)]
    elif kind=="group_user":
        # нужен user_id в Xv: добавим временно через merge из CACHE[cands]
        tmp = Xv.merge(CACHE["cands"][ ["query_id","user_id"] ].drop_duplicates(), on="query_id", how="left")
        users = tmp["user_id"].values
        gkf = GroupKFold(n_splits=n_splits)
        mask_pairs=[]
        for tr, va in gkf.split(uniq_q, groups=tmp.drop_duplicates("query_id")["user_id"].values):
            mask_pairs.append((np.isin(qids, uniq_q[tr]), np.isin(qids, uniq_q[va])))
        return mask_pairs
    else: # time
        tmp = CACHE["qv"][ ["query_id","query_ts"] ].drop_duplicates().sort_values("query_ts")
        chunks = np.array_split(tmp["query_id"].values, n_splits)
        pairs=[]
        for k in range(n_splits):
            va_q = chunks[k]
            tr_q = np.concatenate([chunks[i] for i in range(n_splits) if i!=k])
            pairs.append((np.isin(qids, tr_q), np.isin(qids, va_q)))
        return pairs

btn_folds = W.Button(description="Сформировать фолды", button_style="primary", layout=BTN_LAYOUT)
out_folds = W.Output()

def on_folds(_):
    with out_folds:
        clear_output()
        if DATA["X_val"] is None:
            print("Сначала соберите фичи.")
            return
        pairs = make_folds(CV["CV_KIND"], CV["N_SPLITS"], DATA["X_val"])
        FOLDS["pairs"]=pairs
        sizes = [int(mask_va.sum()) for _, mask_va in pairs]
        print("FOLDS:", len(pairs), "| val sizes:", sizes)

btn_folds.on_click(on_folds)
W.VBox([btn_folds, out_folds])

In [ ]:
def recall_ndcg_at_k(qids, labels, scores, ks=(10,20,50,100)):
    df = pd.DataFrame({"q":qids, "y":labels, "s":scores})
    df = df.sort_values(["q","s"], ascending=[True,False])
    res={}
    for K in ks:
        hits=0; tot=0; dcg=0.0; idcg=0.0
        for q, grp in df.groupby("q", sort=False):
            arr = grp["y"].values[:K]
            hit = int(arr.sum()>0)
            hits += hit; tot += 1
            # NDCG binary: позиция первого релевантного
            nd = 0.0
            for r,v in enumerate(arr, start=1):
                if v>0:
                    nd = 1.0/math.log2(r+1); break
            dcg += nd; idcg += 1.0
        res[f"recall@{K}"]=hits/max(1,tot)
        res[f"ndcg@{K}"]=dcg/max(1.0,idcg)
    return res

In [ ]:
btn_train = W.Button(description="Тренировать ранкер (CV + OOF)", button_style="success", layout=BTN_LAYOUT)
out_train = W.Output()

RUN = dict(oof=None, models=[], fi=None, cv_scores=None)

def train_cv_lgbm(X, y, groups, folds_pairs, feat_cols):
    oof_scores = np.zeros(len(X), dtype="float32")
    models=[]; fold_metrics=[]
    for i,(mask_tr, mask_va) in enumerate(folds_pairs, start=1):
        tr_idx = np.where(mask_tr)[0]; va_idx=np.where(mask_va)[0]
        Xtr, ytr = X.iloc[tr_idx][feat_cols], y.iloc[tr_idx]
        Xva, yva = X.iloc[va_idx][feat_cols], y.iloc[va_idx]
        # группы — длины по query_id
        gtr = X.iloc[tr_idx]["query_id"].value_counts().sort_index().values
        gva = X.iloc[va_idx]["query_id"].value_counts().sort_index().values

        params = dict(
            objective="lambdarank",
            metric="ndcg",
            ndcg_at=[20],
            learning_rate=MODEL["LR"],
            num_leaves=MODEL["NUM_LEAVES"],
            max_depth=MODEL["MAX_DEPTH"],
            min_data_in_leaf=MODEL["MIN_DATA_LEAF"],
            feature_fraction=MODEL["FEATURE_FRACT"],
            bagging_fraction=MODEL["BAGGING_FRACT"],
            bagging_freq=1,
            lambda_l2=MODEL["L2"],
            verbose=-1,
            seed=STATE["RANDOM_SEED"],
            num_threads=STATE["JOBS"] if STATE["JOBS"]>0 else None
        )
        train_set = lgb.Dataset(Xtr, ytr, group=gtr, free_raw_data=True)
        valid_set = lgb.Dataset(Xva, yva, group=gva, reference=train_set, free_raw_data=True)
        model = lgb.train(
            params, train_set,
            num_boost_round=MODEL["N_EST"],
            valid_sets=[valid_set],
            valid_names=["valid"],
            callbacks=[lgb.early_stopping(MODEL["EARLY_ROUNDS"], verbose=False)]
        )
        pred = model.predict(Xva, num_iteration=model.best_iteration)
        oof_scores[va_idx]=pred.astype("float32")
        # метрики
        m = recall_ndcg_at_k(X.iloc[va_idx]["query_id"].values, yva.values, pred, ks=(10,20,50,100))
        fold_metrics.append(m)
        models.append(model)
        print(f"[fold {i}] best_iter={model.best_iteration} | metrics:", {k:round(v,4) for k,v in m.items()})
    # фич-импортансы
    fi = None
    try:
        imp = np.mean([m.feature_importance(importance_type="gain") for m in models], axis=0)
        fi = pd.DataFrame({"feature":feat_cols, "gain":imp}).sort_values("gain", ascending=False)
    except Exception:
        pass
    return oof_scores, models, fi, fold_metrics

def train_cv_catboost(X, y, groups, folds_pairs, feat_cols):
    oof_scores = np.zeros(len(X), dtype="float32")
    models=[]; fold_metrics=[]
    for i,(mask_tr, mask_va) in enumerate(folds_pairs, start=1):
        tr_idx = np.where(mask_tr)[0]; va_idx=np.where(mask_va)[0]
        Xtr, ytr = X.iloc[tr_idx][feat_cols], y.iloc[tr_idx]
        Xva, yva = X.iloc[va_idx][feat_cols], y.iloc[va_idx]
        group_ids_tr = X.iloc[tr_idx]["query_id"].values
        group_ids_va = X.iloc[va_idx]["query_id"].values

        params = dict(
            loss_function="YetiRankPairwise",
            iterations=MODEL["N_EST"],
            learning_rate=MODEL["LR"],
            depth=MODEL["MAX_DEPTH"],
            l2_leaf_reg=MODEL["L2"],
            random_seed=STATE["RANDOM_SEED"],
            verbose=False
        )
        train_pool = Pool(Xtr, label=ytr, group_id=group_ids_tr)
        valid_pool = Pool(Xva, label=yva, group_id=group_ids_va)
        model = CatBoostRanker(**params)
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True, verbose=False)
        pred = model.predict(valid_pool)
        oof_scores[va_idx]=pred.astype("float32")
        m = recall_ndcg_at_k(X.iloc[va_idx]["query_id"].values, yva.values, pred, ks=(10,20,50,100))
        fold_metrics.append(m)
        models.append(model)
        print(f"[fold {i}] metrics:", {k:round(v,4) for k,v in m.items()})
    fi=None
    try:
        imp = np.mean([m.get_feature_importance(type="FeatureImportance", data=Pool(DATA["X_val"][feat_cols], label=DATA["y_val"], group_id=DATA["X_val"]["query_id"].values)) for m in models], axis=0)
        fi = pd.DataFrame({"feature":feat_cols, "gain":imp}).sort_values("gain", ascending=False)
    except Exception:
        pass
    return oof_scores, models, fi, fold_metrics

def on_train(_):
    with out_train:
        clear_output()
        if DATA["X_val"] is None or not FOLDS["pairs"]:
            print("Сначала соберите фичи и сформируйте фолды.")
            return
        Xv, yv = DATA["X_val"], DATA["y_val"]
        feat_cols = DATA["feat_cols"]
        if MODEL["LIB"]=="lightgbm":
            if lgb is None:
                print("LightGBM не установлен")
                return
            oof, models, fi, fold_metrics = train_cv_lgbm(Xv, yv, DATA["group_val"], FOLDS["pairs"], feat_cols)
        else:
            if CatBoostRanker is None:
                print("CatBoost не установлен")
                return
            oof, models, fi, fold_metrics = train_cv_catboost(Xv, yv, DATA["group_val"], FOLDS["pairs"], feat_cols)
        RUN.update(oof=oof, models=models, fi=fi, cv_scores=fold_metrics)
        # Общие OOF-метрики
        oof_metrics = recall_ndcg_at_k(Xv["query_id"].values, yv.values, oof, ks=(10,20,50,100))
        print("OOF:", {k:round(v,4) for k,v in oof_metrics.items()})
        # Сохраняем OOF
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_dir = Path(DATA["meta"]["run_dir"])
        oof_df = Xv[["query_id","item_id"]].copy()
        oof_df["label"]=yv.values
        oof_df["oof_score"]=oof
        oof_df.to_parquet(run_dir/"oof.parquet", index=False)
        if fi is not None:
            fi.to_csv(run_dir/"fi.csv", index=False)
        save_json(run_dir/"cv_metrics.json", dict(folds=RUN["cv_scores"], oof=oof_metrics))
        print("Saved:", run_dir)

btn_train.on_click(on_train)
W.VBox([btn_train, out_train])

In [ ]:
btn_infer = W.Button(description="Инференс на TEST → ranked.parquet", button_style="warning", layout=BTN_LAYOUT)
out_infer = W.Output()

def on_infer(_):
    with out_infer:
        clear_output()
        if DATA["X_test"] is None or not RUN["models"]:
            print("Нет тестовых фич или не обучены модели.")
            return
        Xt = DATA["X_test"]
        feat_cols = DATA["feat_cols"]
        # усредним предсказания по фолдам
        preds = np.zeros(len(Xt), dtype="float32")
        for m in RUN["models"]:
            p = m.predict(Xt[feat_cols]) if MODEL["LIB"]=="lightgbm" else m.predict(Pool(Xt[feat_cols], group_id=Xt["query_id"].values))
            preds += p.astype("float32")
        preds /= max(1,len(RUN["models"]))
        ranked = Xt[["query_id","item_id"]].copy()
        ranked["score"]=preds
        ranked = ranked.sort_values(["query_id","score"], ascending=[True,False]).groupby("query_id").head(STATE["K_OUT"])
        ranked["rank"] = ranked.groupby("query_id")["score"].rank(ascending=False, method="first").astype("int32")
        run_dir = Path(DATA["meta"]["run_dir"])
        ranked.to_parquet(run_dir/"ranked.parquet", index=False)
        # (опц.) сохранить все скоры
        Xt.assign(score=preds).to_parquet(run_dir/"test_scores.parquet", index=False)
        print("✓ saved:", run_dir/"ranked.parquet")

btn_infer.on_click(on_infer)
W.VBox([btn_infer, out_infer])

In [ ]:
btn_slices = W.Button(description="OOF срезы и сводка", button_style="", layout=BTN_LAYOUT)
out_slices = W.Output()

def on_slices(_):
    with out_slices:
        clear_output()
        if RUN["oof"] is None:
            print("Сначала обучи модель, чтобы получить OOF.")
            return
        Xv, yv, oof = DATA["X_val"], DATA["y_val"], RUN["oof"]
        # общие метрики
        base = recall_ndcg_at_k(Xv["query_id"].values, yv.values, oof, ks=(10,20,50,100))
        print("OOF:", {k:round(v,4) for k,v in base.items()})
        # холодные срезы
        if "is_cold_item" in DATA["X_val"].columns:
            mask = DATA["X_val"]["is_cold_item"]==1
            m_cold = recall_ndcg_at_k(Xv["query_id"].values[mask.values], yv.values[mask.values], oof[mask.values], ks=(20,))
            m_warm = recall_ndcg_at_k(Xv["query_id"].values[~mask.values], yv.values[~mask.values], oof[~mask.values], ks=(20,))
            print("cold_item@20:", m_cold, "| warm_item@20:", m_warm)
        # сохраним короткий отчёт
        run_dir = Path(DATA["meta"]["run_dir"])
        save_json(run_dir/"oof_report.json", dict(overall=base))
        print("Report saved:", run_dir/"oof_report.json")

btn_slices.on_click(on_slices)
W.VBox([btn_slices, out_slices])

In [ ]:
btn_pass = W.Button(description="Сохранить ranker_report.json", button_style="success", layout=BTN_LAYOUT)
out_pass = W.Output()

def on_pass(_):
    with out_pass:
        clear_output()
        run_dir = Path(DATA["meta"]["run_dir"])
        oof_metrics = {}
        try:
            oof_metrics = json.loads((run_dir/"cv_metrics.json").read_text())["oof"]
        except Exception:
            pass
        report = dict(
            dataset_id = STATE["DATASET_ID"],
            profile    = STATE["PROFILE_PATH"],
            run_id     = DATA["meta"]["run_id"],
            params = dict(CV=CV, MODEL=MODEL, FEATS=FEATS, K_IN=STATE["K_IN"], K_OUT=STATE["K_OUT"], seed=STATE["RANDOM_SEED"]),
            files = dict(
                X_val=str(run_dir/"X_rank_val.parquet"),
                X_test=str(run_dir/"X_rank_test.parquet"),
                oof=str(run_dir/"oof.parquet"),
                ranked=str(run_dir/"ranked.parquet")
            ),
            oof_metrics = oof_metrics,
            saved_at = datetime.utcnow().isoformat()+"Z"
        )
        save_json(run_dir/"ranker_report.json", report)
        print("✓ saved:", run_dir/"ranker_report.json")
        print(json.dumps(report["oof_metrics"], indent=2))

btn_pass.on_click(on_pass)
W.VBox([btn_pass, out_pass])

In [ ]:
btn_panic = W.Button(description="⚡ PANIC: минимальный набор фич + быстрый ранкер", button_style="danger", layout=BTN_LAYOUT)
out_panic = W.Output()

def on_panic(_):
    with out_panic:
        clear_output()
        # жёсткие настройки
        FEATS.update(use_source_feats=True, use_pop_recency=True, use_user_hist=False, use_user_item_match=True,
                     use_item_meta=False, use_context=False, use_cold_flags=True, windows_days=[7,30], user_last_k=3)
        MODEL.update(LIB="lightgbm", N_EST=250, LR=0.07, MAX_DEPTH=6, NUM_LEAVES=63, EARLY_ROUNDS=30, FEATURE_FRACT=0.9, BAGGING_FRACT=0.8, MIN_DATA_LEAF=20, L2=1.0)
        print("PANIC config applied. Rebuild feats → Train → Infer.")
        # перезапустить сборку фич, обучение и инференс
        on_build_feats(None)
        on_folds(None)
        on_train(None)
        on_infer(None)

btn_panic.on_click(on_panic)
W.VBox([btn_panic, out_panic])

In [ ]:
btn_all = W.Button(description="🚀 Быстрый прогон: check → preagg → feats → folds → train → infer → report", button_style="success", layout=BTN_LAYOUT)
out_all = W.Output()

def on_all(_):
    with out_all:
        clear_output()
        try:
            on_check(None)
            on_preagg(None)
            on_build_feats(None)
            on_folds(None)
            on_train(None)
            on_infer(None)
            on_slices(None)
            on_pass(None)
            print("\n✓ Готово.")
        except Exception as e:
            print("ОШИБКА:", e)
            raise

btn_all.on_click(on_all)
W.VBox([btn_all, out_all])

In [ ]:
btn_list = W.Button(description="Показать файлы ранкера", button_style="", layout=BTN_LAYOUT)
out_list = W.Output()

def on_list(_):
    with out_list:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        d = A["ranker"]
        if not d.exists():
            print("Папка ещё не создана:", d); return
        files=[]
        for p in sorted(d.rglob("*")):
            if p.is_file():
                try:
                    size = p.stat().st_size/1024/1024
                except Exception:
                    size = float("nan")
                files.append({"path":str(p), "size_MB": round(size,3)})
        df = pd.DataFrame(files)
        display(df if len(df) else "файлов пока нет")

btn_list.on_click(on_list)
W.VBox([btn_list, out_list])

### Пороги и подсказки
- **Цель**: Recall@20 ≥ 0.90 (OOF). В цейтноте (PANIC) — ≥ 0.88.
- Если много пустых запросов в ранк-входе → вернись в 02 и усили fallbacks.
- Холодные просадки → добавь контент/brand-aware фичи, расширь окна covis (в 02 и здесь в user_hist).
- Если OOF ≫ holdout → уменьши глубину/усиль регуляризацию.